In [1]:
BLOCKSIZE = 16

def Unpad(s: bytes, blocksize = BLOCKSIZE):
    assert type(s) is bytes and len(s) % blocksize == 0
    last = s[-1]
    assert all( char == last for char in s[-last:] )
    return s[:-last]

def Pad(s: bytes, blocksize = BLOCKSIZE):
    assert type(s) is bytes
    npad = blocksize - len(s) % blocksize
    s += bytes( [npad] * npad )
    assert len(s) % blocksize == 0
    return s

In [2]:
Pad(b'Hello world')

b'Hello world\x05\x05\x05\x05\x05'

In [3]:
ORACLE_KEY = b'\xe7\xef\x1e\x7f\xd7\x87\xa4\xeb\x10<\xd9\x9f\x8b\xec\x03\x8f'
ORACLE_INITIALIZATION = b'\x04\xcfNb\x1eV4d\xbb\xb7)\xee\x94@\xe6\xa3'

def Oracle(s: bytes):
    key = ORACLE_KEY
    initialization = ORACLE_INITIALIZATION
    assert type(s) is bytes
    prefix = b"comment1=cooking%20MCs;userdata="
    suffix = b";comment2=%20like%20a%20pound%20of%20bacon"
    plaintext = prefix + s.replace(b';', b'\;').replace(b'=', b'\=') + suffix 
    assert b';admin=true;' not in plaintext
    
    return EncryptCBC(Pad(plaintext), key, initialization)

def IsAdmin(ciphertext):
    key = ORACLE_KEY
    initialization = ORACLE_INITIALIZATION
    plaintext = DecryptCBC(ciphertext, key, initialization)
    return b';admin=true;' in plaintext

def DecryptOracle(ciphertext: bytes): # This is just to view and verify the final result
    key = ORACLE_KEY
    initialization = ORACLE_INITIALIZATION
    plaintext = DecryptCBC(ciphertext, key, initialization)
    return plaintext

In [4]:
from Crypto.Cipher import AES

def XOR(X: bytes, Y: bytes) -> bytes:
    assert type(X) is bytes and type(Y) is bytes and len(X) == len(Y)
    return bytes([ x^y for x, y in zip(X, Y) ])

def Chunkerize(x, chunksize, strict = True):
    x = list(x)
    assert len(x) % chunksize == 0 if strict else True
    for n in range( len(x) // chunksize ):
        yield x[ n*chunksize : (n+1)*chunksize ]

def EncryptCBC(plaintext, key, initialization):
    cipher = AES.new(key, AES.MODE_CBC, iv = initialization)
    ciphertext = cipher.encrypt(Pad(plaintext))
    return ciphertext

def DecryptCBC(ciphertext, key, initialization):
    cipher = AES.new(key, AES.MODE_CBC, iv = initialization)
    plaintext = cipher.decrypt(ciphertext)
    return Unpad(plaintext)

In [5]:
from random import choices

key = bytes( choices(range(256), k = 16) )
initialization = bytes( choices(range(256), k = 16) )

plaintext = b'Cookin MCs like a pound of '
ciphertext = EncryptCBC(plaintext, key, initialization)
DecryptCBC(ciphertext, key, initialization)

b'Cookin MCs like a pound of '

In [6]:
# Ok, so I'm going to assume the attacker knows the prefix and suffix values...
# Nothing in the instructions says the attacker can't know this
# and I don't see how this could be possible otherwise

prefix = b"comment1=cooking%20MCs;userdata="
suffix = b";comment2=%20like%20a%20pound%20of%20bacon"

prefix_len = len(prefix)
suffix_len = len(suffix)

assert prefix_len % BLOCKSIZE == 0

In [7]:
# I want to get something to decrypt to ;admin=true;
# but the oracle will escape any ; and = characters I try to pass in so I'll replace them wil nulls
# then try to manipulate those nulls back into ; and = characters

plaintext_block = b'A'*BLOCKSIZE + b';admin=true;'.replace(b';', b'\0').replace(b'=', b'\0')
ciphertext = Oracle(plaintext_block) 
ciphertext

b'\x17\xfbFl~\x93\x1a\xce\xc9\xde\xbe\x90e3\xff\x06\x88\xbd\x05\xef\xc4q9\xb3\xc4op9\x93:\x16\x96\x0e8\x17\xb1\x80<\x96\xadT\xc3E1\x16\n\xe9\t\xb2V\xd3x\xd1:|\x7fui\xeeL\xdb\xfcsK\x98\xfeC\x94\x87\xc2\xa7~sO\xb0F\x9e\r\xa6\xb3\xa2\x12[\x8c/\x0f\x93\xea\xe0\xc9Ba\xa3\x1a\xbc{@\x9e\x8d\x15\xb5\xd0\xfdU\xe7\xca\x96 \x1a\x1f\xa6]2\x9ePC\x97H\x80\x8c\x07\x80\xcd\xf5C\t\xd2\x8a'

In [8]:
from itertools import product as CartesianProduct

for n, [byte1, byte2, byte3] in enumerate(CartesianProduct(range(0xFF), range(0xFF), range(0xFF))):
    print(n) if n % 1e6 == 0 else None
    new_ciphertext = list(ciphertext)
    variable_block = new_ciphertext[ prefix_len : prefix_len+BLOCKSIZE ]
    # Positions 0, 6, and 11 of my ciphertext contain the nulls that I am trying to turn into ; and =
    variable_block[0] = byte1
    variable_block[6] = byte2
    variable_block[11] = byte3
    new_ciphertext[ prefix_len : prefix_len+BLOCKSIZE ] = variable_block
    if IsAdmin(bytes(new_ciphertext)):
        print(byte1, byte2, byte3)
        break

0
1000000
2000000
3000000
53 171 10


In [9]:
DecryptOracle(bytes(new_ciphertext))

b'comment1=cooking%20MCs;userdata=Y\xd4\xfb\x03\xc3P\x8e\x90\x96\xe211\x89\x00\x07\x94;admin=true;;comment2=%20like%20a%20pound%20of%20bacon\n\n\n\n\n\n\n\n\n\n'